# Градієнтний спуск

## Генеруємо дані

In [248]:
import numpy as np

sample_np = np.random.rand(100, 2)
sample_np.shape

(100, 2)

## Створюємо поліном

In [249]:
from sympy import symbols


def polynomial():
    x1 = symbols('x1')
    x2 = symbols('x2')
    return 4*x1**2 + 5*x2**2 + 2*x1*x2 + 3*x1 - 6*x2

## Функція простого градієнтного спуску

In [250]:
from sklearn.preprocessing import PolynomialFeatures
from sympy import Poly, lambdify


def polynomial_regression_gradient_descent(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int):
    # Визначаємо степінь полінома з символьного виразу
    ploinomial_rate = Poly(poloinomial_func).total_degree()

    # Готуємо ціль як значення полінома на даних
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # Узгоджено генеруємо поліноміальні ознаки у train
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # Ключова правка: довжина theta дорівнює кількості колонок у X_poly
    theta = np.zeros(p, dtype=float)

    for _ in range(epochs):
        y_pred = X_poly @ theta          # (m,)
        error = y_pred - y               # (m,)
        gradient = (2.0 / m) * (X_poly.T @ error)  # (p,)
        theta -= learning_rate * gradient

    return theta

## Функція градієнтного спуску методом SGD

In [251]:
def polynomial_regression_SGD(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int, shuffle: bool = True):
    # 1) Цільові значення з символічного полінома
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # 2) Поліноміальні ознаки узгоджено з іншими функціями
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # 3) Ініціалізація ваг
    theta = np.zeros(p, dtype=float)

    # 4) Класичний SGD: обхід по зразках
    for _ in range(epochs):
        if shuffle:
            perm = np.random.permutation(m)
            X_poly_epoch = X_poly[perm]
            y_epoch = y[perm]
        else:
            X_poly_epoch = X_poly
            y_epoch = y

        for i in range(m):
            xi = X_poly_epoch[i]          # (p,)
            yi = y_epoch[i]               # scalar
            y_pred_i = xi @ theta         # scalar
            error_i = y_pred_i - yi       # scalar
            grad_i = 2.0 * xi * error_i   # (p,)
            theta -= learning_rate * grad_i

    return theta


## Функція градієнтного спуску методом RMSPROP

In [252]:
def polynomial_regression_rmsprop(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int, gamma = 0.9, epsilon = 1e-8):
    # 1) Цільові значення з символічного полінома
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # 2) Поліноміальні ознаки узгоджено з іншими функціями (include_bias=True)
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # 3) Ініціалізація ваг та накопичувача квадратів градієнтів
    theta = np.zeros(p, dtype=float)
    eg2 = np.zeros(p, dtype=float)

    # 4) RMSProp із повним батчем
    for _ in range(epochs):
        y_pred = X_poly @ theta                 # (m,)
        error = y_pred - y                      # (m,)
        grad = (2.0 / m) * (X_poly.T @ error)   # (p,)

        eg2 = gamma * eg2 + (1.0 - gamma) * (grad ** 2)     # (p,)
        theta -= (learning_rate / (np.sqrt(eg2) + epsilon)) * grad

    return theta

## Функція градієнтного спуску методом ADAM

In [253]:
def polynomial_regression_adam(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int, beta1=0.9, beta2=0.999, epsilon = 1e-8):
    # 1) Цільові значення з символічного полінома
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # 2) Поліноміальні ознаки узгоджено з іншими функціями (include_bias=True)
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # 3) Ініціалізація ваг та накопичувача квадратів градієнтів
    theta = np.zeros(p, dtype=float)
    eg2 = np.zeros(p, dtype=float)
    es = np.zeros(p, dtype=float)
    steps = 0

    for _ in range(epochs):
        y_pred = X_poly @ theta
        error = y_pred - y
        loss = np.mean(error**2)
        grad = (2.0 / m) * (np.transpose(X_poly) @ error)
        steps += 1

        eg2 = beta2 * eg2 + (1.0 - beta2) * (grad ** 2)
        es = beta1 * es + (1.0 - beta1) * grad

        es_hat = es / (1.0 - beta1 ** steps)
        eg2_hat = eg2 / (1.0 - beta2 ** steps)

        theta -= (learning_rate / (np.sqrt(eg2_hat) + epsilon)) * es_hat

    return theta


## Функція градієнтного спуску методом NADAM

In [254]:
def polynomial_regression_nadam(features: np.ndarray, poloinomial_func, learning_rate: float, epochs: int, beta1=0.9, beta2=0.999, epsilon = 1e-8):
    # 1) Цільові значення з символічного полінома
    x1, x2 = symbols('x1 x2')
    f = lambdify((x1, x2), poloinomial_func, modules='numpy')
    y = f(features[:, 0], features[:, 1]).astype(float).reshape(-1)  # (m,)

    # 2) Поліноміальні ознаки узгоджено з іншими функціями (include_bias=True)
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(features)  # (m, p)
    m, p = X_poly.shape

    # 3) Ініціалізація ваг та моментів
    theta = np.zeros(p, dtype=float)
    v = np.zeros(p, dtype=float)   # eg2
    m1 = np.zeros(p, dtype=float)  # es
    steps = 0

    for _ in range(epochs):
        y_pred = X_poly @ theta
        error = y_pred - y
        grad = (2.0 / m) * (X_poly.T @ error)
        steps += 1

        # Оновлення моментів
        v = beta2 * v + (1.0 - beta2) * (grad ** 2)
        m1 = beta1 * m1 + (1.0 - beta1) * grad

        # Bias-corrected
        m1_hat = m1 / (1.0 - beta1 ** steps)
        v_hat = v / (1.0 - beta2 ** steps)

        # Nadam "погляд уперед" без подвійної корекції
        nesterov = beta1 * m1_hat + (1.0 - beta1) * (grad / (1.0 - beta1 ** steps))

        theta -= (learning_rate / (np.sqrt(v_hat) + epsilon)) * nesterov

    return theta

## Визначаємо функцію прогнозування

In [255]:
def predict(X_input: np.ndarray, poloinomial_func, theta: np.ndarray):
    # Та сама логіка PolynomialFeatures, що і в train
    ploinomial_rate = Poly(poloinomial_func).total_degree()
    poly = PolynomialFeatures(degree=ploinomial_rate, include_bias=True)
    X_poly = poly.fit_transform(X_input)  # Порядок і набір ознак детерміновані degree і n_features
    return X_poly @ theta



## Визначаємо функцію, що генерує значення, застосовуючи поліном (реальні значення)

In [256]:
def actual_vals(X_input: np.ndarray, poloinomial_func):
    x1, x2 = symbols('x1 x2')
    polynomial = lambdify((x1, x2), poloinomial_func, modules='numpy')
    return polynomial(X_input[:, 0], X_input[:, 1])

## Застосовуємо прогнозування з функціями градієнтного спуску

In [257]:
test_np = np.random.rand(10, 2)
pol_func = polynomial()
pred_gd = predict(test_np, pol_func, polynomial_regression_gradient_descent(sample_np, pol_func, 0.1, 10000))
pred_gd

array([-1.57033404,  5.19398438,  1.54102856,  3.92213147,  0.9914736 ,
       -0.93754121,  1.37694618,  2.15019474,  0.35300574, -1.08066852])

In [258]:
pred_rmsprop = predict(test_np, pol_func, polynomial_regression_rmsprop(sample_np, pol_func, 0.02, 10000))
pred_rmsprop

array([-1.59382322,  5.16088754,  1.50476893,  3.88190488,  0.95671894,
       -0.95989257,  1.34213973,  2.11259041,  0.32025665, -1.10682657])

In [259]:
pred_sgd = predict(test_np, pol_func, polynomial_regression_SGD(sample_np, pol_func, 0.1, 10000))
pred_sgd

array([-1.571137  ,  5.19255161,  1.5405709 ,  3.92654129,  0.98910946,
       -0.93997871,  1.37311636,  2.14998247,  0.3502397 , -1.08289839])

In [260]:
pred_adam = predict(test_np, pol_func, polynomial_regression_adam(sample_np, pol_func, 0.02, 10000))
pred_adam

array([-1.571137  ,  5.19255161,  1.5405709 ,  3.92654129,  0.98910946,
       -0.93997871,  1.37311636,  2.14998247,  0.3502397 , -1.08289839])

In [261]:
pred_nadam = predict(test_np, pol_func, polynomial_regression_nadam(sample_np, pol_func, 0.02, 10000))
pred_nadam

array([-1.571137  ,  5.1925516 ,  1.54057089,  3.92654128,  0.98910945,
       -0.93997871,  1.37311635,  2.14998246,  0.35023969, -1.08289839])

## Обчислюємо реальний результат

In [262]:
actual = actual_vals(test_np, pol_func)
actual

array([-1.571137  ,  5.19255161,  1.5405709 ,  3.92654129,  0.98910946,
       -0.93997871,  1.37311636,  2.14998247,  0.3502397 , -1.08289839])

## Вимірюємо час виконання різних функцій графієнтного спуску

In [263]:
%%timeit -r 3
predict(test_np, pol_func, polynomial_regression_gradient_descent(sample_np, pol_func, 0.1, 10000))

28.7 ms ± 547 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [264]:
%%timeit -r 3
predict(test_np, pol_func, polynomial_regression_rmsprop(sample_np, pol_func, 0.02, 10000))

53 ms ± 453 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [265]:
%%timeit -r 3
predict(test_np, pol_func, polynomial_regression_SGD(sample_np, pol_func, 0.1, 10000))

2.19 s ± 3.32 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [266]:
%%timeit -r 3
predict(test_np, pol_func, polynomial_regression_adam(sample_np, pol_func, 0.02, 10000))

102 ms ± 938 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [267]:
%%timeit -r 3
predict(test_np, pol_func, polynomial_regression_nadam(sample_np, pol_func, 0.02, 10000))

89.1 ms ± 589 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)


## Висновок

У цій вправі ми використали наступні методи градієнтного спуску:

- Звичайний градієнтний спуск
- SGD
- RMSProp
- ADAM
- NADAM

Найточніший прогноз дали **SGD** та **ADAM**. За часом виконання найшвидшим виявився звичайний градієнтний спуск (хоча і найменш точний), а найповільнішим - **SGD**.